<a href="https://colab.research.google.com/github/acl-jianghao/Langchain-Scripts/blob/main/04_conversation_history_cache.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

conversation

In [314]:
!pip install -Uqq OpenAI langchain tiktoken colab-env pymupdf pinecone-client
!pip install -Uqq ndg-httpsclient pyopenssl pyasn1 colab-env pymupdf chromadb

In [243]:
import os
import colab_env

In [244]:
!wget https://progforperf.github.io/Expert_C_Programming.pdf -O /content/gdrive/MyDrive/data/pdf/c.pdf
!wget https://www.halvorsen.blog/documents/programming/python/resources/Python%20Programming.pdf -O /content/gdrive/MyDrive/data/pdf/python.pdf

--2023-07-25 07:44:38--  https://progforperf.github.io/Expert_C_Programming.pdf
Resolving progforperf.github.io (progforperf.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to progforperf.github.io (progforperf.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2251803 (2.1M) [application/pdf]
Saving to: ‘/content/gdrive/MyDrive/data/pdf/c.pdf’

/content/gdrive/MyD 100%[===================>]   2.15M  --.-KB/s    in 0.06s   

2023-07-25 07:44:38 (33.8 MB/s) - ‘/content/gdrive/MyDrive/data/pdf/c.pdf’ saved [2251803/2251803]

--2023-07-25 07:44:38--  https://www.halvorsen.blog/documents/programming/python/resources/Python%20Programming.pdf
Resolving www.halvorsen.blog (www.halvorsen.blog)... 46.250.210.141
Connecting to www.halvorsen.blog (www.halvorsen.blog)|46.250.210.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3463729 (3.3M) [application/pdf]
Saving to: ‘/content/gdrive/M

In [245]:
import openai
from langchain.document_loaders import DirectoryLoader, PyMuPDFLoader

loader = DirectoryLoader('/content/gdrive/MyDrive/data/pdf/', glob="**/*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()


text_splitter
- CharacterTextSplitter:separator > len
- RecursiveCharacterTextSplitter: (support metadata)
- TokenTextSplitter

In [246]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
recur_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)

In [247]:
# split_docs = text_splitter.split_documents(docs)
split_docs = recur_splitter.split_documents(docs)

print(len(split_docs))
print(len(split_docs[4].page_content))

964
927


In [248]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

import pinecone

In [249]:
openai.api_key=os.getenv('OPENAI_API_KEY')

In [250]:
embeddings = OpenAIEmbeddings()

In [251]:
pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),  # find at app.pinecone.io
    environment=os.getenv('PINECONE_ENV'),  # next to api key in console
)

pinecone_indexes = pinecone.list_indexes()
if 'conversation-pdf' in pinecone_indexes:
    print("do something like: pinecone.delete_index('conversation-pdf'), and re-create")
else:
    pinecone.create_index('conversation-pdf', dimension=1536, metric="cosine")
db_pinecone = Pinecone.from_documents(split_docs, embeddings, index_name='conversation-pdf')


do something like: pinecone.delete_index('conversation-pdf'), and re-create


In [252]:
# questions = [
#     "what is C programming language?",
#     "what is point in C?",
#     "what is decorator in Python?",
# ]

# chat_history = []

In [ ]:
# answer_docs = db_pinecone.similarity_search(question[0])
# answer_docs

In [253]:
import functools
from langchain.callbacks import get_openai_callback

def track_token_usage(chain, query):
    with get_openai_callback() as cb:
        result = chain(query)
        print(cb)
        print("\n")
    return result


# def token_usage():
#     def decorate(func):
#         @functools.wraps(func)
#         def wrapper(*args, **kw):
#             result = strack_token_usage(func, *args, **kw)
#             return result
#         # wrapper.__name__ = func.__name__
#         # wrapper.__doc__ = func.__doc__
#         return wrapper
#     return decorate

In [254]:
# from langchain.retrievers import PineconeHybridSearchRetriever
# from pinecone_text.sparse import BM25Encoder

retriever = db_pinecone.as_retriever()

In [255]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo', verbose=False)

In [285]:
questions = [
    "what is C programming language?",
    "what did I just ask?",
]

chat_history = []

In [334]:
def conversation(chain, questions):
    for question in questions:
        input = {'question': question, 'chat_history': chat_history}
        result = chain(input)
        # result = track_token_usage(chain, input)
        chat_history.append((question, result['answer']))
        print("Question:\n %s \n" % question)
        print("Answer:\n %s \n" % result['answer'])
        print("---------------------------------- \n")

Conversation based on local PDF

In [337]:
from langchain.chains import ConversationalRetrievalChain

conversation_chain = ConversationalRetrievalChain.from_llm(
                        llm=llm,
                        retriever=retriever,
                        verbose=True,
                    )

In [ ]:
conversation(conversation_chain, questions)

Conversation based on local PDF and memory
- Open issue: https://github.com/langchain-ai/langchain/issues/2303

In [339]:
from langchain.memory import ConversationBufferMemory

# Open issue: https://github.com/langchain-ai/langchain/issues/2303

chat_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

conversation_chain_buffer_memory = ConversationalRetrievalChain.from_llm(
        llm,
        retriever=retriever,
        # return_source_documents=True,
        verbose=False,
        memory=chat_memory,
        # output_key='answer',
        # combine_docs_chain_kwargs={'prompt': prompt},
        # condense_question_llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    )


for question in questions:
    input = {'question': question, 'chat_history': chat_history}
    result = conversation_chain_buffer_memory(input)
    chat_history.append((question, result['answer']))
    print("Question:\n %s \n" % question)
    print("Answer:\n %s \n" % result['answer'])
    print("---------------------------------- \n")


print(conversation_chain_buffer_memory.memory.buffer)


# transform = TransformChain(
#     input_variables=["input"],
#     output_variables=["question", "chat_history"],
#     transform = lambda inputs: {
#         "question": inputs["input"],
#         "chat_history": [
#             (human.group(1).strip(), ai.group(1).strip())
#             for human, ai in zip(
#                 re.compile(r'Human: (.*(?:\n(?!(Human|AI):).*)*)').finditer(inputs["history"]),
#                 re.compile(r'AI: (.*(?:\n(?!(Human|AI):).*)*)').finditer(inputs["history"]),
#             )
#         ],
#     },
# )

# conversational_retrieval = ConversationalRetrievalChain.from_llm(
#     llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
#     retriever=retriever,
# )

# chat = SequentialChain(
#     memory=ConversationBufferMemory(),
#     input_variables=["input"],
#     output_variables=["answer"],
#     chains=[transform, conversational_retrieval],
# )



# for question in questions:
#     input = {'question': question, 'chat_history': chat_history}
#     result = chat.run(question)
#     # chat_history.append((question, result['answer']))
#     print("Question:\n %s \n" % question)
#     print("Answer:\n %s \n" % result)
#     print("---------------------------------- \n")


Question:
 what is C programming language? 

Answer:
 C programming language is a high-level programming language that was developed in the early 1970s. It was originally designed for system programming and has since become one of the most widely used programming languages. C is known for its simplicity, efficiency, and flexibility, making it suitable for a wide range of applications. It allows programmers to have direct control over the hardware and memory, making it ideal for developing operating systems, embedded systems, and other low-level applications. C has influenced many other programming languages and is often used as a foundation for learning other languages. 

---------------------------------- 

Question:
 what did I just ask? 

Answer:
 I'm sorry, but I don't have access to the previous question. 

---------------------------------- 

[HumanMessage(content='what is C programming language?', additional_kwargs={}, example=False), AIMessage(content='C programming language is

Conversation with local PDF and summary
- havn't supportted

Conversation cache

In [ ]:
from langchain.memory import ConversationBufferMemory

# Open issue: https://github.com/langchain-ai/langchain/issues/2303

chat_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

conversation_chain_buffer_memory = ConversationalRetrievalChain.from_llm(
        llm,
        retriever=retriever,
        # return_source_documents=True,
        verbose=False,
        memory=chat_memory,
        # output_key='answer',
        # combine_docs_chain_kwargs={'prompt': prompt},
        # condense_question_llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    )


for question in questions:
    input = {'question': question, 'chat_history': chat_history}
    result = conversation_chain_buffer_memory(input)
    chat_history.append((question, result['answer']))
    print("Question:\n %s \n" % question)
    print("Answer:\n %s \n" % result['answer'])
    print("---------------------------------- \n")


print(conversation_chain_buffer_memory.memory.buffer)
